In [1]:
from imports import *
from functions import *
from auth import *
from constants import * 
# Activer l'auto-rechargement des modules dès qu'ils sont modifiés
%load_ext autoreload
%autoreload 2

In [2]:
job_wanted = "ingeniero de datos junior"
location_searched = None

In [3]:
driver = create_driver()
login(driver, email, password)

Clicked on User infos !


In [4]:
search_for_jobs(driver, job_wanted,location_searched)

Overlay supprimé avec succès


In [5]:
# GET OFFERS OF THE PAGE
try:
    offers = find_offers_div(driver)
    
except Exception as e:
    print(str(e))

20 offres trouvées.


One Page

In [ ]:
##########################################################################################################################################
list_offers =[]
count_salaries = 0 
for offer in offers:
    offer_dict = {}

    if web_search_url is not None:
        print(f"Web Search URL trouvé !\n{web_search_url}")
        web_search_url = driver.get(web_search_url)

    try:
        title = driver.find_element(By.CSS_SELECTOR, ".js-o-link.fc_base").text
        offer_dict["Title"] = title
        print(f"Titre trouvé: {title}")  # Affichage avec i+1
    except:
        offer_dict["Title"] = None
        print(f"Titre non trouvé")  # Affichage avec i+1


    try:
        company = driver.find_element(By.CSS_SELECTOR, ".fc_base.t_ellipsis").text
        offer_dict["Company"] = company
        print(f"Entreprise trouvée: {company}")  # Affichage avec i+1
    except:
        offer_dict["Company"] = None
        print(f"Entreprise non trouvée")  # Affichage avec i+1

    try:

        location = driver.find_element(By.CSS_SELECTOR, "p.fs16.mb5").text

        offer_dict["Location"] = location

        print(f"Lieu trouvé: {location}")  # Affichage avec i+1
    except:
        offer_dict["Location"] = None
        print(f"Lieu non trouvé")  # Affichage avec i+1

    # Extraction du salaire
    try:
        salary = offer.find_element(By.XPATH, './/span[contains(@class, "i_salary")]/parent::span').text.strip()

        if salary != "":
            print(salary)
            count_salaries += 1

    except:
        offer_dict["Salary"] = None
        print("Erreur lors de la récupération du salaire")

    # Extraction du type de contrat
    try:
        contract_type = driver.find_elements(By.CSS_SELECTOR, "p.dFlex.mb10")[1].text  # Le contrat est le 2ème paragraphe
        offer_dict["Contract Type"] = contract_type
        print(f"Type de contrat trouvé: {contract_type}")
    except:
        offer_dict["Contract Type"] = None
        print("Type de contrat non trouvé")

    # Extraction du temps de travail
    try:
        work_time = driver.find_elements(By.CSS_SELECTOR, "p.dFlex.mb10")[2].text  # Le temps de travail est le 3ème paragraphe
        offer_dict["Work Time"] = work_time
        print(f"Temps de travail trouvé: {work_time}")
    except:
        offer_dict["Work Time"] = None
        print("Temps de travail non trouvé")

    # Extraction du lieu de travail
    try:
        work_location = driver.find_elements(By.CSS_SELECTOR, "p.dFlex.mb10")[3].text  # Le lieu de travail est le 4ème paragraphe
        offer_dict["Work Location"] = work_location
        print(f"Lieu de travail trouvé: {work_location}")
    except:
        offer_dict["Work Location"] = None
        print("Lieu de travail non trouvé")


    # Verified or not
    try :
        verified = driver.find_element(By.XPATH, "//p[@class='dFlex vm_fx fs16']").text
        print(verified)
        offer_dict["Verified"] = verified
    except Exception as e:
        verified = False
        offer_dict["Verified"] = verified
        print(str(e))

    try:
        description = driver.find_element(By.XPATH, "//div[@class='fs16 t_word_wrap']").text
        offer_dict["Description"] = description
        print("Description du poste trouvée")
    except Exception as e:
        print(str(e))

    # Extraire le contenu de la description
    try:
        acerca_de_la_compania = driver.find_element(By.CSS_SELECTOR, "p[show-more]").text
        offer_dict["About the company"] = acerca_de_la_compania
        print(f"Description de la compagnie trouvée :\n{acerca_de_la_compania}")
    except:
        acerca_de_la_compania = None
        print("Description non trouvée")

    try:
        # Find company evaluation 
        company_rating = driver.find_element(By.XPATH, "//p[@class='fs50 fwB lh1 tc']").text
        offer_dict["Company Rating"] = company_rating
        print(f"{company} Rating:\n{company_rating}/5")
    except Exception as e:
        print("Company Rating non trouvé")

    try:
        # Récupérer le texte brut de l'évaluation
        evaluation_number = driver.find_element(By.XPATH, "//p[@class='fc_aux mt5 fs16']").text

        print(f"Texte trouvé : {evaluation_number}")

        # Regex pour capturer un nombre au format "12.665"
        match = re.search(r"\d{1,3}(?:\.\d{1,3})*", evaluation_number)

        if match:
            rating_number = match.group(0)
            offer_dict["Ratings Number"] = rating_number
            print(f"{company} Rating Number : {rating_number}")
        else:
            offer_dict["Ratings Number"] = None
            print("Aucune note trouvée dans le texte")

    except Exception as e:
        offer_dict["Ratings Number"] = None
        print(f"Erreur lors de la récupération du rating : {e}")
    
    offer_dict['ID'] = ulid.new()

    list_offers.append(offer_dict)

print(f"Salaries found: {count_salaries}")

df_offers = pd.DataFrame(list_offers)
df_offers.head()

Several Pages

In [6]:
while True:
    offers = driver.find_elements(By.CSS_SELECTOR, "article[data-id]")

    for offer in offers:
        offer_dict = {}

        if web_search_url is not None:
            print(f"Web Search URL trouvé !\n{web_search_url}")
            web_search_url = driver.get(web_search_url)

        try:
            title = driver.find_element(By.CSS_SELECTOR, ".js-o-link.fc_base").text
            offer_dict["Title"] = title
            print(f"Titre trouvé: {title}")
        except:
            offer_dict["Title"] = None
            print("Titre non trouvé")

        try:
            company = driver.find_element(By.CSS_SELECTOR, ".fc_base.t_ellipsis").text
            offer_dict["Company"] = company
            print(f"Entreprise trouvée: {company}")
        except:
            offer_dict["Company"] = None
            print("Entreprise non trouvée")

        try:
            location = driver.find_element(By.CSS_SELECTOR, "p.fs16.mb5").text
            offer_dict["Location"] = location
            print(f"Lieu trouvé: {location}")
        except:
            offer_dict["Location"] = None
            print("Lieu non trouvé")

        try:
            salary = offer.find_element(By.XPATH, './/span[contains(@class, "i_salary")]/parent::span').text.strip()
            if salary != "":
                print(salary)
                count_salaries += 1
        except:
            offer_dict["Salary"] = None
            print("Erreur lors de la récupération du salaire")

        try:
            contract_type = driver.find_elements(By.CSS_SELECTOR, "p.dFlex.mb10")[1].text
            offer_dict["Contract Type"] = contract_type
            print(f"Type de contrat trouvé: {contract_type}")
        except:
            offer_dict["Contract Type"] = None
            print("Type de contrat non trouvé")

        try:
            work_time = driver.find_elements(By.CSS_SELECTOR, "p.dFlex.mb10")[2].text
            offer_dict["Work Time"] = work_time
            print(f"Temps de travail trouvé: {work_time}")
        except:
            offer_dict["Work Time"] = None
            print("Temps de travail non trouvé")

        try:
            work_location = driver.find_elements(By.CSS_SELECTOR, "p.dFlex.mb10")[3].text
            offer_dict["Work Location"] = work_location
            print(f"Lieu de travail trouvé: {work_location}")
        except:
            offer_dict["Work Location"] = None
            print("Lieu de travail non trouvé")

        try:
            verified = driver.find_element(By.XPATH, "//p[@class='dFlex vm_fx fs16']").text
            print(verified)
            offer_dict["Verified"] = verified
        except Exception as e:
            verified = False
            offer_dict["Verified"] = verified
            print(str(e))

        try:
            description = driver.find_element(By.XPATH, "//div[@class='fs16 t_word_wrap']").text
            offer_dict["Description"] = description
            print("Description du poste trouvée")
        except Exception as e:
            print(str(e))

        try:
            acerca_de_la_compania = driver.find_element(By.CSS_SELECTOR, "p[show-more]").text
            offer_dict["About the company"] = acerca_de_la_compania
            print(f"Description de la compagnie trouvée :\n{acerca_de_la_compania}")
        except:
            acerca_de_la_compania = None
            print("Description non trouvée")

        try:
            company_rating = driver.find_element(By.XPATH, "//p[@class='fs50 fwB lh1 tc']").text
            offer_dict["Company Rating"] = company_rating
            print(f"{company} Rating:\n{company_rating}/5")
        except Exception as e:
            print("Company Rating non trouvé")

        try:
            evaluation_number = driver.find_element(By.XPATH, "//p[@class='fc_aux mt5 fs16']").text
            print(f"Texte trouvé : {evaluation_number}")
            match = re.search(r"\d{1,3}(?:\.\d{1,3})*", evaluation_number)
            if match:
                rating_number = match.group(0)
                offer_dict["Ratings Number"] = rating_number
                print(f"{company} Rating Number : {rating_number}")
            else:
                offer_dict["Ratings Number"] = None
                print("Aucune note trouvée dans le texte")
        except Exception as e:
            offer_dict["Ratings Number"] = None

        offer_dict['ID'] = ulid.new()
        list_offers.append(offer_dict)

    if not go_to_next_page_with_siguiente(driver):
        break

print(f"Salaries found: {count_salaries}")

df_offers = pd.DataFrame(list_offers)
df_offers.head()

Titre trouvé: Work From Home Data Engineer (Junior) / Ref. 0960
Entreprise trouvée: BairesDev LLC
Lieu trouvé: Pasto, Nariño
Erreur lors de la récupération du salaire
Type de contrat trouvé: Tiempo Completo
Temps de travail trouvé: Remoto
Lieu de travail non trouvé
Empresa verificada
Description du poste trouvée
Description de la compagnie trouvée :
BairesDev® es una compañía líder de servicios tecnológicos que ofrece soluciones de software para empresas de gran reconocmiento de la industria como Google, Rolls-Royce y Pinterest y las startups de mayor potencial. Nuestro equipo diverso, conformado por más de 4.000 profesionales que trabajan de manera totalmente remota desde más de 50 países, sigue creciendo.

A lo largo de los años, BairesDev ha colaborado en proyectos innovadores que benefician a usuarios alrededor del mundo y nos...
BairesDev LLC Rating:
2,33/5
Texte trouvé : 184 Evaluaciones
BairesDev LLC Rating Number : 184
Titre trouvé: Work From Home Data Engineer (Junior) / Ref. 

,Title,Company,Location,Salary,Contract Type,Work Time,Work Location,Verified,Description,About the company,Company Rating,Ratings Number,ID
0,Work From Home Data Engineer (Junior) / Ref. 0960,BairesDev LLC,"Pasto, Nariño",NaN,Tiempo Completo,Remoto,None,Empresa verificada,"At BairesDev®, weve been leading the way in te...",BairesDev® es una compañía líder de servicios ...,"2,33",184,01JS7J12BEQZ59E86VXMK6SFAM
1,Work From Home Data Engineer (Junior) / Ref. 0960,BairesDev LLC,"Pasto, Nariño",NaN,Tiempo Completo,Remoto,None,Empresa verificada,"At BairesDev®, weve been leading the way in te...",BairesDev® es una compañía líder de servicios ...,"2,33",184,01JS7J14E9CSNTTVPFJH63HHP3
2,Work From Home Data Engineer (Junior) / Ref. 0960,BairesDev LLC,"Pasto, Nariño",NaN,Tiempo Completo,Remoto,None,Empresa verificada,"At BairesDev®, weve been leading the way in te...",BairesDev® es una compañía líder de servicios ...,"2,33",184,01JS7J16G7AZHEFQMTMAVGFKQQ
3,Work From Home Data Engineer (Junior) / Ref. 0960,BairesDev LLC,"Pasto, Nariño",NaN,Tiempo Completo,Remoto,None,Empresa verificada,"At BairesDev®, weve been leading the way in te...",BairesDev® es una compañía líder de servicios ...,"2,33",184,01JS7J18KJWGH3ZSQE5HXT9P9A
4,Work From Home Data Engineer (Junior) / Ref. 0960,BairesDev LLC,"Pasto, Nariño",NaN,Tiempo Completo,Remoto,None,Empresa verificada,"At BairesDev®, weve been leading the way in te...",BairesDev® es una compañía líder de servicios ...,"2,33",184,01JS7J1APNJMNH8DTYH2F117TN


In [7]:
if location_searched is not None:
    output_file_name = f"computrabajo_offers_{job_wanted.replace(" ", "_")}_{location_searched}.csv"
else:
    output_file_name = f"computrabajo_offers_{job_wanted.replace(" ", "_")}.csv"

In [8]:
df_offers.shape[0]

72

In [ ]:
# # Vérifier si le fichier existe déjà et le supprimer si nécessaire
# if os.path.exists(output_file_name):
#     os.remove(output_file_name)

# Sauvegarder le DataFrame en CSV avec encodage UTF-8
df_offers.to_csv(output_file_name, index=False, encoding='latin-1', errors='ignore')

In [ ]:
while True:
    try:
        current_url = driver.current_url

        # Si l'URL contient déjà ?p=, on incrémente le numéro
        match = re.search(r'[?&]p=(\d+)', current_url)
        if match:
            page_number = int(match.group(1)) + 1
            new_url = re.sub(r'(p=)\d+', f'p={page_number}', current_url)
            print(f"Nouvel URL (avec pagination) : {new_url}")
            driver.get(new_url)
            close_banner(driver)
            close_overlay(driver)
            list_url.append(new_url)

        else:
            print("URL sans paramètre 'p=', clic sur 'Siguiente'")
            list_url.append(current_url)

            siguiente = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//span[@title='Siguiente']"))
            )
            siguiente.click()
            close_banner(driver)
            close_overlay(driver)

    except Exception as e:
        print(f"Fin ou erreur : {e}")
        break

In [ ]:
# Cherche le conteneur principal des offres
offers_container = driver.find_element(By.ID, "offersGridOfferContainer")
if offers_container:
    # Cherche tous les articles dont l'ID commence par quelque chose (à préciser si besoin)
    offers = offers_container.find_elements(By.XPATH, ".//article[starts-with(@id, '')]")
    
    if offers:
        print(f"{len(offers)} offres trouvées.")
    else:
        print("Aucune offre trouvée dans le conteneur.")
else:
    print("Conteneur 'offersGridOfferContainer' introuvable.")

###################################################################################################################################################

list_offers = []

# Initialisation des compteurs pour chaque type de donnée
count = {
    "Title": {"found": 0, "not_found": 0},
    "Company": {"found": 0, "not_found": 0},
    "Location": {"found": 0, "not_found": 0},
    "Salary": {"found": 0, "not_found": 0},
    "Contract Type": {"found": 0, "not_found": 0},
    "Time Schedule": {"found": 0, "not_found": 0},
    "Work Place": {"found": 0, "not_found": 0},
    "Verified": {"found": 0, "not_found": 0},
    "Description": {"found": 0, "not_found": 0},
    "About the Company": {"found": 0, "not_found": 0},
    "Company Rating": {"found": 0, "not_found": 0},
    "Evaluations Numbers": {"found": 0, "not_found": 0}
}

for i in range(len(offers)):
    try:
        offer = offers[i]

        # Scroll jusqu'à l'offre pour la rendre visible
        driver.execute_script("arguments[0].scrollIntoView(true);", offer)
        time.sleep(0.5)

        # Attente explicite avant de cliquer sur l'offre
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(offer))
        offer.click()
        print(f"[{i+1}] Clic réussi")  # Affichage avec i+1
        time.sleep(random.uniform(1, 2))

        offer_dict = {}

        try:
            title = driver.find_element(By.CSS_SELECTOR, ".js-o-link.fc_base").text
            offer_dict["Title"] = title
            count["Title"]["found"] += 1
            print(f"[{i+1}] Titre trouvé: {title}")  # Affichage avec i+1
        except:
            offer_dict["Title"] = None
            count["Title"]["not_found"] += 1
            print(f"[{i+1}] Titre non trouvé")  # Affichage avec i+1

        try:
            company = driver.find_element(By.CSS_SELECTOR, ".fc_base.t_ellipsis").text
            offer_dict["Company"] = company
            count["Company"]["found"] += 1
            print(f"[{i+1}] Entreprise trouvée: {company}")  # Affichage avec i+1
        except:
            offer_dict["Company"] = None
            count["Company"]["not_found"] += 1
            print(f"[{i+1}] Entreprise non trouvée")  # Affichage avec i+1

        try:
            location = driver.find_element(By.CSS_SELECTOR, ".mr10").text
            offer_dict["Location"] = location
            count["Location"]["found"] += 1
            print(f"[{i+1}] Lieu trouvé: {location}")  # Affichage avec i+1
        except:
            offer_dict["Location"] = None
            count["Location"]["not_found"] += 1
            print(f"[{i+1}] Lieu non trouvé")  # Affichage avec i+1

        # Recherche des autres détails sans 'box_detail'
        four_details = driver.find_elements(By.CSS_SELECTOR, ".dFlex.mb10")

        if len(four_details) >= 4:
            offer_dict["Salary"] = four_details[0].text
            offer_dict["Contract Type"] = four_details[1].text
            offer_dict["Time Schedule"] = four_details[2].text
            offer_dict["Work Place"] = four_details[3].text

            count["Salary"]["found"] += 1
            count["Contract Type"]["found"] += 1
            count["Time Schedule"]["found"] += 1
            count["Work Place"]["found"] += 1
            print(f"[{i+1}] Détails trouvés: Salaire, Type de contrat, Horaires, Lieu de travail")  # Affichage avec i+1
        else:
            print(f"[{i+1}] ❌ Détails manquants")  # Affichage avec i+1
            count["Salary"]["not_found"] += 1
            count["Contract Type"]["not_found"] += 1
            count["Time Schedule"]["not_found"] += 1
            count["Work Place"]["not_found"] += 1

        # Vérification du statut de vérification
        try:
            verified = driver.find_element(By.CSS_SELECTOR, ".dFlex.vm_fx.fs16").text
        except:
            verified = "unverified"
        offer_dict["verified"] = verified
        count["Verified"]["found"] += 1
        print(f"[{i+1}] Statut de vérification: {verified}")  # Affichage avec i+1

        # Informations complémentaires
        try:
            description = driver.find_element(By.CSS_SELECTOR, ".fs16.t_word_wrap").text
            offer_dict["Description"] = description
            count["Description"]["found"] += 1
            print(f"[{i+1}] Description trouvée")  # Affichage avec i+1
        except:
            offer_dict["Description"] = None
            count["Description"]["not_found"] += 1
            print(f"[{i+1}] Description non trouvée")  # Affichage avec i+1

        try:
            about_company = driver.find_element(By.CSS_SELECTOR, ".fs18.mb20.fwB").text
            offer_dict["About the Company"] = about_company
            count["About the Company"]["found"] += 1
            print(f"[{i+1}] À propos de l'entreprise trouvé")  # Affichage avec i+1
        except:
            offer_dict["About the Company"] = None
            count["About the Company"]["not_found"] += 1
            print(f"[{i+1}] À propos de l'entreprise non trouvé")  # Affichage avec i+1

        try:
            company_rating = driver.find_element(By.CSS_SELECTOR, ".fs50.fwB.lh1.tc").text
            offer_dict["Company Rating"] = company_rating
            count["Company Rating"]["found"] += 1
            print(f"[{i+1}] Note de l'entreprise trouvée: {company_rating}")  # Affichage avec i+1
        except:
            offer_dict["Company Rating"] = None
            count["Company Rating"]["not_found"] += 1
            print(f"[{i+1}] Note de l'entreprise non trouvée")  # Affichage avec i+1

        # Extraction du nombre d’évaluations
        try:
            evaluations_text = driver.find_element(By.CSS_SELECTOR, ".fc_aux.mt5.fs16").text
            match = re.search(r'\d+\s*,\s*\d+|\d+', evaluations_text)
            if match:
                cleaned = match.group().replace(' ', '').replace(',', '.')
                offer_dict["Evaluations Numbers"] = float(cleaned)
                count["Evaluations Numbers"]["found"] += 1
                print(f"[{i+1}] Nombre d'évaluations trouvé: {cleaned}")  # Affichage avec i+1
            else:
                offer_dict["Evaluations Numbers"] = None
                count["Evaluations Numbers"]["not_found"] += 1
                print(f"[{i+1}] Nombre d'évaluations non trouvé")  # Affichage avec i+1

        except:
            offer_dict["Evaluations Numbers"] = None
            count["Evaluations Numbers"]["not_found"] += 1
            print(f"[{i+1}] Nombre d'évaluations non trouvé")  # Affichage avec i+1

        offer_dict["unique_key"] = str(ulid.new())
        list_offers.append(offer_dict)

        print(f"[{i+1}] ✅ Offre ajoutée")  # Affichage avec i+1

        time.sleep(1)

    except Exception as e:
        print(f"[{i+1}] ❌ Erreur : {e}")  # Affichage avec i+1

# Affichage des totaux pour chaque type de donnée
for key, value in count.items():
    print(f"\n{key}: Trouvé {value['found']} / Non trouvé {value['not_found']}")


In [ ]:
def collect_offers(driver):
    offers = []
    close_banner(driver)
    try:
        jobs_div = driver.find_elements(By.XPATH, "//article[@class='box_offer']")

        for job in jobs_div:
            close_banner(driver)
            offer_dict = {}

            try:
                offer_date_elem = job.find_element(By.CLASS_NAME, "fc_aux")
                offer_date_text = offer_date_elem.text.strip().lower()

                if "ayer" in offer_date_text:
                    offer_date = today - timedelta(days=1)
                elif offer_date_text.startswith("hace") and "horas" in offer_date_text:
                    offer_date = today
                elif offer_date_text.startswith("hace") and "días" in offer_date_text:
                    date_diff = re.search(r'\d+', offer_date_text)
                    offer_date = today - timedelta(days=int(date_diff.group())) if date_diff else today
                else:
                    continue

                offer_dict['offer_date'] = offer_date.strftime('%Y-%m-%d')
            except:
                continue  # skip this job if date is missing or broken

            # Génère une clé unique
            offer_dict['unique_key'] = str(ulid.new())

            try:
                title_elem = job.find_element(By.CLASS_NAME, "title_offer")
                offer_dict['title'] = title_elem.text.strip()
            except:
                offer_dict['title'] = ""

            # Clique pour charger les détails du job
            try:
                job.click()
                time.sleep(1)
            except:
                pass

            try:
                company_elem = driver.find_element(By.CSS_SELECTOR, 'a.dIB.mr10')
                offer_dict['company'] = company_elem.text.strip()
            except:
                offer_dict['company'] = ""
            try:

                location = driver.find_element(By.CSS_SELECTOR, ".fs16.mb5")
                offer_dict['location'] = location.text.strip()


            except:
                offer_dict['location'] = ""

            try:
                contract_section = driver.find_element(By.XPATH, "//div[@class='mbB']")
                parts = contract_section.find_elements(By.CLASS_NAME, "dFlex")

                if len(parts) >= 3:
                    offer_dict['salary'] = parts[0].text.strip()
                    offer_dict['duration'] = parts[1].text.strip()
                    offer_dict['schedule'] = parts[2].text.strip()
                else:
                    offer_dict['salary'] = offer_dict['duration'] = offer_dict['schedule'] = ""
            except:
                offer_dict['salary'] = offer_dict['duration'] = offer_dict['schedule'] = offer_dict['location'] = ""

            try:
                description_elem = driver.find_element(By.CLASS_NAME, "t_word_wrap")
                offer_dict['description'] = description_elem.text.strip()
            except:
                offer_dict['description'] = ""

            offer_dict['url'] = driver.current_url

            offers.append(offer_dict)

    except Exception as e:
        print(f"Erreur lors de la récupération des détails :\n{e}")

    return offers

def export_to_csv(offers, filename, keywords, location):
    # dynamically build the filename according to the provided keywords and location and today date

    if location is not None:
        filename = f"{today.strftime('%Y-%m-%d')}_{keywords}_{location}.csv"
    else: 
        filename = f"{today.strftime('%Y-%m-%d')}_{keywords}.csv"
        
    # get the offers file and detect the keys put them in a list
    keys = offers[0].keys()
    file = open(filename, 'w', newline='')
    # write columns names equal to the keys
    writer = csv.DictWriter(file, fieldnames=keys)
    # write the header of the csv file
    writer.writeheader()
    # write each offer in a new row
    for offer in offers:
        writer.writerow(offer)






    # We use the csv.DictWriter class from the csv module in Python
    import csv
    
    writer = csv.DictWriter(file, fieldnames=['unique_key', 'offer_date', 'title', 'company', 'description', 'url','salary', 'duration','schedule', 'location'])


def job_search_and_collect_offers(driver, keywords, location=None):
            search_for_jobs(driver, keywords, location)
            offers = collect_offers(driver)
            # get the date of today in string format with the location and keyword, then build the file name dynamically
            filename = f"{today.strftime('%Y-%m-%d')}_{keywords}_{location}.csv"
            export_to_csv(offers, filename, keywords, location)

In [ ]:
try:
    job_search_and_collect_offers(driver, "ingeniero de datos junior")
except Exception as e:
    print(f"Erreur lors de la recherche et collecte des offres :\n{e}")

In [ ]:
Dataclass = dataclasses.dataclass

@Dataclass
class Offer:
    title: str
    company: Optional[str] = None
    verified : bool = False
    location : str
    contract_type: str
    schedule: str
    location: str
    salary: Optional[int] = None